In [10]:
#Задание 1

#Напишите функцию, которая возвращает сумму элементов на диагонали. Т. е. 13+32+23+35.

data = [
    [13, 25, 23, 34],
    [45, 32, 44, 47],
    [12, 33, 23, 95],
    [13, 53, 34, 35],
]

def diagonal_sum(data):
    """
    Принимает на вход список списков с равным кол-вом столбцов и строк.
    Возвращает сумму элементов по диагонали
    diagonal_sum(data)
    103
    """
    i = 0
    sum = 0
    while i < len(data):
        sum += data[i][i]
        i += 1
    return sum

diagonal_sum(data)

103

In [10]:
#Задание 2
#Дан список чисел, часть из которых имеют строковый тип или содержат буквы.
#Напишите функцию, которая возвращает сумму квадратов элементов, которые могут быть числами.

data = [1, '5', 'abc', 20, '2']

def sum_square(data_list):
    """
    Принимает на вход список чисел, часть из которых имеют строковый тип или содержат буквы.
    Возвращает сумму квадратов элементов, которые являются числами и 
    тех элементов, которые могут быть числами
    
    sum_square([1, '5', 'abc', 20, '2'])
    430.0
    """
    i = 0
    while i < len(data_list):
        try:
            data_list[i] = float(data_list[i])**2
            i += 1
        except ValueError:
            del data_list[i]
            continue 
            
    return sum(data_list)


sum_square(data)

430.0

In [4]:

#Задание 3
#Напишите функцию, которая возвращает название валюты (поле 'Name')
#с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js
import requests

def exchange_rates():
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']


# требуется учитывать не только само значение валюты на данном ресурсе ('Value'), но и в каком номинале она там
# присутствует. То есть требуется поделить значение на номинал. В противном случае, например, мы получим для Юаней
# значение около 97. Но реальный курс юаня к рублю примерно 9.7. Дело в номинале. Он равен на данном ресурсе для юаня 10 и тд. 


def name_highest_exchange_rate(exchange_rates):
    """
    Возвращает название валюты, которой реальный курс к рублю наибольший
    
    """
    maximum = 0
    
    for value in exchange_rates.values():
        real_value = float(value['Value'])/float(value['Nominal']) #делим на номинал для получения реального курса валюты к рублю
        
        if real_value > maximum:
            maximum = real_value
            name = value['Name']

    return name

name_highest_exchange_rate(exchange_rates())

'СДР (специальные права заимствования)'

In [84]:
#Задание 4
#Последнее упражнение с занятия
#1. Добавьте в класс еще один формат, который возвращает название валюты (например, 'Евро').

#2. Добавьте в класс параметр diff (со значениями True или False),
#который в случае значения True в методах eur и usd будет возвращать не курс валюты,
#а изменение по сравнению в прошлым значением.

class Rate:

    def __init__(self, format='value'):
        self.format = format
        
    diff = False
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self, diff):
        """
        Возвращает курс евро на сегодня в формате self.format
        или разницу между курсами сегодня и вчера при условии если параметр diff равен True
        
        Rate().eur(True)
        0.2687999999999988
        
        Rate().eur(False)
        75.6562
        """
        if diff == True:
            return exchange_rates()['EUR']['Value'] - exchange_rates()['EUR']['Previous']
        else:
            return self.make_format('EUR')
    
    def usd(self, diff):
        """
        Возвращает курс евро на сегодня в формате self.format
        или разницу между курсами сегодня и вчера при условии, если параметр diff равен True
        
        Rate().usd(True)
        0.15460000000000207
        
        Rate().usd(False)
        66.4467
        """
        if diff == True:
            return exchange_rates()['USD']['Value'] - exchange_rates()['USD']['Previous']
        else:
            return self.make_format('USD')
    
    def name_format(self, currency):
        """
        Возвращает название валюты
        
        Rate('name').name_format('USD') 
        'Доллар США'
        """
        response = self.exchange_rates()
        if self.format == 'name':
            return response[currency]['Name']
    
Rate().eur(False)

75.6562

In [86]:
#Задание 5
#Напишите функцию, возвращающую сумму первых n чисел Фибоначчи

import matplotlib.pyplot as plt
import math

n = int(input('Введите число n: ', ))


def fib_func(n):
    """
    (Здесь имеем ввиду, что первые два члена ряда есть 1 и 1)
    
    Принимает на вход натуральное число. 
    Возвращает сумму ряда Фибонначи и сам ряд соотв. этому нат. числу.
    
    fib_func(7)
    33 [1, 1, 2, 3, 5, 8, 13]
    
    fib_func(1)
    1 [1]
    """
    fib1 = 1 
    fib2 = 1
    fib_n_list = []
    if n == 0:
        sum_n = 0
    elif n == 1:
        sum_n = fib1
        fib_n_list = [fib1]
    else:
        sum_n = fib1 + fib2
        fib_n_list = [fib1, fib2]
        i = 2
        while i < n:
            fib_n = fib1 + fib2 
            fib1 = fib2
            fib2 = fib_n
            sum_n += fib_n
            fib_n_list.append(fib_n)
            i += 1
    return sum_n, fib_n_list

print('Сумма первых {} чисел Фибонначи - {} (учитывая что первые два числа это 1 и 1), сам ряд Фибонначи {}'
      .format(n, fib_func(n)[0], fib_func(n)[1]))

Введите число n: 22
Сумма первых 22 чисел Фибонначи - 46367 (учитывая что первые два числа это 1 и 1), сам ряд Фибонначи [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711]


In [87]:
#Задание 6
#Напишите функцию, преобразующую произвольный список вида ['2018-01-01', 'yandex', 'cpc', 100]
#в словарь {'2018-01-01': ##{'yandex': {'cpc': 100}}}

list_a = ['2018-01-01', 'yandex', 'cpc', 100] # пример списка №1
list_b = list_a[::-1]*2  #пример списка №2


def list_in_dict(list_x):
    """
    Принимает на вход список. Возвращает вложанный словарь вида:
    
    ['2018-01-01', 'yandex', 'cpc', 100]
    {'2018-01-01': {'yandex': {'cpc': 100}}}
    """
    i = 2
    dict_x = list_x[-1]
    while i <= len(list_x):
        dict_x = {list_x[-i]:dict_x}
        i += 1
    return dict_x


print('Из списка {} получаем словарь ---> {}'.format(list_a, list_in_dict(list_a)))
print('Из списка {} получаем словарь ---> {}'.format(list_b, list_in_dict(list_b)))

Из списка ['2018-01-01', 'yandex', 'cpc', 100] получаем словарь ---> {'2018-01-01': {'yandex': {'cpc': 100}}}
Из списка [100, 'cpc', 'yandex', '2018-01-01', 100, 'cpc', 'yandex', '2018-01-01'] получаем словарь ---> {100: {'cpc': {'yandex': {'2018-01-01': {100: {'cpc': {'yandex': '2018-01-01'}}}}}}}
